# UHF SCF

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## Molecular setup

UHF SCF calculations start off in the same way as RHF SCF calculations. We first need a second-quantized Hamiltonian that expresses all the integrals in the AO basis.

In [4]:
molecule = gqcpy.Molecule.HRingFromDistance(3, 1.0, 0)  # create a neutral molecule
N = molecule.numberOfElectrons()
N_alpha = N//2 + 1
N_beta = N//2

spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
S = spinor_basis.quantizeOverlapOperator().parameters()

sq_hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule)  # 'sq' for 'second-quantized'

In [5]:
print(molecule)

Number of electrons: 3 
H  (0.57735, 0, 0)
H  (-0.288675, 0.5, 0)
H  (-0.288675, -0.5, 0)



## Plain UHF SCF

Then, we need an UHF SCF solver and an UHF SCF environment.

In [6]:
environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)
plain_solver = gqcpy.UHFSCFSolver.Plain(threshold=1.0e-04, maximum_number_of_iterations=1000)  # the system is not converging very rapidly

Using GQCP's capabilities of 'injecting' Python code inside the C++ library, we'll add an intermediary step that doesn't modify the environment, but just prints out the current Fock matrices.

This is done by writing a `Python` function that manipulates the solver's environment (of type `EnvironmentType`), and subsequently wrapping that function into a `FunctionalStep_EnvironmentType`. The instance of type `FunctionalStep_EnvironmentType` is then subsequently inserted inside the algorithm that should be performed.

In this example, we'll print out the alpha- and beta- Fock matrices.

In [7]:
def print_fock_matrices(environment):
    if (environment.fock_matrices_alpha):
        print("Alpha Fock matrix:\n{}\n".format(environment.fock_matrices_alpha[-1]))
        print("Beta Fock matrix:\n{}".format(environment.fock_matrices_beta[-1]))
    else:
        print("No Fock matrices in the queue.")

print_step = gqcpy.FunctionalStep_UHFSCFEnvironment(print_fock_matrices)

In [8]:
plain_solver.insert(print_step, index=0)  # insert at the front

In [9]:
qc_structure = gqcpy.UHF.optimize(plain_solver, environment)

TypeError: 'method' object is not subscriptable

In [10]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

NameError: name 'qc_structure' is not defined

In [11]:
uhf_parameters = qc_structure.groundStateParameters()

NameError: name 'qc_structure' is not defined

In [12]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.alpha))

NameError: name 'uhf_parameters' is not defined

In [13]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.beta))

NameError: name 'uhf_parameters' is not defined

In [14]:
print(plain_solver.numberOfIterations())

0


## DIIS UHF SCF

In [15]:
environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, sq_hamiltonian, S)
diis_solver = gqcpy.UHFSCFSolver.DIIS(threshold=1.0e-04, maximum_number_of_iterations=1000, minimum_subspace_dimension=6, maximum_subspace_dimension=6)  # the system is not converging very rapidly

In [16]:
qc_structure = gqcpy.UHF.optimize(diis_solver, environment)

In [17]:
print(qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value())

-0.6305219488655234


In [18]:
uhf_parameters = qc_structure.groundStateParameters()

In [19]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.alpha))

[-1.033  0.189  0.767]


In [20]:
print(uhf_parameters.orbitalEnergies(gqcpy.Spin.beta))

[-0.889  0.818  0.939]


In [21]:
print(diis_solver.numberOfIterations())

25
